In [3]:
#import pacakges 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import math
import numpy
import cv2
from pylab import *
from numpy import *
from numpy import array
import random;
from scipy.interpolate import RectBivariateSpline

# Use Sift to get matching point 

In [ ]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16
#!pip install opencv-python
import cv2
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt
img1 = cv2.imread('./Images/dest.png')
img2 = cv2.imread('./Images/src.png')
#plt.imshow(img1),plt.show(block=False)
gray= cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2= cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
plt.imshow(img1),plt.show(block=False)
#plt.close()


In [ ]:
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()
kp1, src = sift.detectAndCompute(gray2,None)
kp2, dst = sift.detectAndCompute(gray,None)
print ("Found keypoints in image 2: " + str(len(kp1)))
print ("Found keypoints in image 1: " + str(len(kp2)))

In [ ]:
bf = cv2.BFMatcher()
matches = bf.match(src, dst)
keypoints = [kp1,kp2]
correspondenceList = []
#matches = np.array(matches)
for match in matches:
            (x1, y1) = keypoints[0][match.queryIdx].pt
            (x2, y2) = keypoints[1][match.trainIdx].pt
            correspondenceList.append([x1, y1, x2, y2])

corrs = np.array((correspondenceList))
estimation_thresh = 0.4
print(len(corrs))

#  Ransac function to get the best homography

In [ ]:
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    randomFour = np.zeros((4,4)) 
    for i in range(1000):
        #find 4 random points to calculate a homography
        point1 = corr[random.randrange(0, len(corr))]
        randomFour [0]=point1
        point2 = corr[random.randrange(0, len(corr))]
        randomFour [1]=point2
        point3 = corr[random.randrange(0, len(corr))]
        randomFour [2]=point3
        point4 = corr[random.randrange(0, len(corr))]
        randomFour [3]=point4

        #call the homography function on those points
        h = findHomography(randomFour)
        if h is  None:
            print("found a singular matrix but ignored it")
            continue
        else:
            inliers = []

        #for every point in the corres
            for i in range(len(corr)):
                d = geometricDistance(corr[i], h)
                if d < 5:
                    inliers.append(corr[i])

            if len(inliers) > len(maxInliers):
                maxInliers = inliers
                finalH = h
           # print ("size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

            if len(maxInliers) > (len(corr)*0.4):
                break
    return finalH, maxInliers

In [4]:
def split_array(corr):
    x=[]
    y=[]
    xdash=[]
    ydash=[]
    b=[]
    for i in range(4):
        x.append(corr[i][0])
        y.append(corr[i][1])
        xdash.append(corr[i][2])
        ydash.append(corr[i][3])
        b.append(corr[i][2])
        b.append(corr[i][3])
    return x,y,xdash,ydash,b

In [ ]:
def geometricDistance(correspondence, h):

    p1 = np.array([correspondence[0], correspondence[1], 1])
    estimatep2 = np.dot(h, p1)
    estimatep2 = correct(estimatep2)

    p2 = np.array([correspondence[2], correspondence[3], 1])
    error = p2 - estimatep2
    return np.linalg.norm(error)

In [6]:
def correct(A):
    if A[2]!=1:
        A[0] = (A[0]/A[2])
        A[1] = (A[1]/A[2])
        A[2] = (A[2]/A[2])
    return A
def round_arr(A):
    A[0]= round(A[0])
    A[1]= round(A[1])
    A[2]= round(A[2])
    return A  

In [5]:
def findHomography (corr):
    x,y,xdash,ydash,b = split_array(corr)
    
    M = np.array([
          [x[0],y[0],1,0,0,0,-x[0]*xdash[0], -y[0]*xdash[0]],
          [0,0,0,x[0],y[0],1,-x[0]*ydash[0], -y[0]*ydash[0]],
          [x[1],y[1],1,0,0,0,-x[1]*xdash[1],-y[1]*xdash[1]],
          [0,0,0,x[1],y[1],1,-x[1]*ydash[1],-y[1]*ydash[1]],
          [x[2],y[2],1,0,0,0,-x[2]*xdash[2],-y[2]*xdash[2]],
          [0,0,0,x[2],y[2],1,-x[2]*ydash[2],-y[2]*ydash[2]],
          [x[3],y[3],1,0,0,0,-x[3]*xdash[3],-y[3]*xdash[3]],
          [0,0,0,x[3],y[3],1,-x[3]*ydash[3],-y[3]*ydash[3]]
      ])
   # print("M = " ,M)
    try:
        a = np.dot(np.linalg.inv(M),b)
        a = np.append(a,1)
        return a.reshape(3,3)
    except:
        a = None
        return None

In [ ]:
#Run the ransac
finalH, inliers = ransac(corrs, estimation_thresh)

print ("Final homography: ", finalH)
print ("Final inliers count: ", len(inliers))
h = np.array(finalH)

if len(inliers) < (0.3*len(corrs)):
    print("not optimal ")

# 1) Use ginput to find correspondances

In [7]:
def getPoints(image):
    img1 = Image.open(image) 
    plt.figure(1)
    plt.imshow(img1)
    print("Please click") 
    x = plt.ginput(4)
    return x

In [8]:
pts_src = getPoints('./Images/image1.jpg')
print(pts_src)
pts_dst=getPoints('./Images/image2.jpg')
print(pts_dst)
plt.close()

Please click


/usr/lib/python3/dist-packages/matplotlib/backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


[(886.8847694471623, 323.45719178082203), (1018.8735170009783, 331.8109099804306), (780.7925483121329, 538.9831213307241), (907.7690649461839, 541.4892367906067)]
Please click
[(441.6315894080235, 302.5728962818005), (560.2543878424657, 317.609589041096), (324.67953461350294, 505.5682485322897), (445.80844850782773, 508.07436399217227)]


In [9]:
corrs = np.concatenate((pts_dst,pts_src),axis=1)
corrs.shape

(4, 4)

In [10]:
#check sizes of image one and image 2
img2 = array(Image.open('./Images/image2.jpg'))
print(img2.shape)
img1 = array(Image.open('./Images/image1.jpg'))
print (img1.shape)

(683, 1024, 3)
(683, 1024, 3)


#

In [11]:
#get x,y,x',y' arrays from cores
def getArrays(src,dest):
    x=[]
    y=[]
    xdash=[]
    ydash=[]

    for i in range(4):
        x.append(src[i][0])
        y.append(src[i][1])
        xdash.append(dest[i][0])
        ydash.append(dest[i][1])

    return x,y,xdash,ydash

In [12]:
def split_array(corr):
    x=[]
    y=[]
    xdash=[]
    ydash=[]
    b=[]
    for i in range(4):
        x.append(corr[i][0])
        y.append(corr[i][1])
        xdash.append(corr[i][2])
        ydash.append(corr[i][3])
        b.append(corr[i][2])
        b.append(corr[i][3])
    return x,y,xdash,ydash,b

In [13]:
h=findHomography(corrs)
print("calculated homography", h)
h, status = cv2.findHomography(np.array(pts_dst),np.array(pts_src))
print("actual homogrpahy", h)

calculated homography [[ 6.98500909e-01  1.91198014e-02  4.57640128e+02]
 [-1.39065914e-01  8.93365222e-01  7.26306503e+01]
 [-2.55503830e-04 -5.55423473e-05  1.00000000e+00]]
actual homogrpahy [[ 6.98500651e-01  1.91202221e-02  4.57640096e+02]
 [-1.39065903e-01  8.93365607e-01  7.26305084e+01]
 [-2.55504204e-04 -5.55418468e-05  1.00000000e+00]]


# Warping the source image to the destination image

## Forward warping 

In [14]:
def Get_max_min_Axis (A,B,C,D):
    
    min_x = min(A[0],B[0],C[0],D[0])
    max_x = max(A[0],B[0],C[0],D[0])
    min_y = min(A[1],B[1],C[1],D[1])
    max_y = max(A[1],B[1],C[1],D[1])
   
    return min_x , max_x , min_y , max_y

In [15]:
#### take into consideration that for homogrpahy calculations [x,y,1] is actually placed as [y,x] in the 2D matrix.
def Get_new_boundaries (h,N,M):
    A= [0,0,1]
    A_new  = np.dot(h,A)
    A_new = correct(A_new)
    A_new = round_arr(A_new)
    A_new[1] = get_coor(A_new,img1.shape[0])
    print("old value of A = ",A,"new value = ",A_new)
    B=[0,N-1,1]
    B_new = np.matmul(h,B )
    B_new = correct(B_new)
    B_new = round_arr(B_new)
    B_new[1] = get_coor(B_new,img1.shape[0])
    C=[M-1,0,1]
    print("old value of B = ",B,"new value = ",B_new)
    C_new = np.matmul(h,C )
    C_new = correct(C_new)
    C_new = round_arr(C_new)
    C_new[1] = get_coor(C_new,img1.shape[0])
    D=[M-1,N-1,1]
    print("old value of C= ",C,"new value = ",C_new)
    D_new = np.matmul(h,D )
    D_new = correct(D_new)
    D_new = round_arr(D_new)
    D_new[1] = get_coor(D_new,img1.shape[0])
    print("old value of D = ",D,"new value = ",D_new)
    return A_new,B_new,C_new,D_new

In [16]:
def get_coor(point,M):
    if point[1]<0:
        point[1]=0
    if point[1]>M:
        point[1]=M-1
    return point[1]
    

In [17]:
print("Fed to homography and resulted from homography:")
print('------------------------------------------------')
A,B,C,D = Get_new_boundaries(h,img2.shape[0],img2.shape[1])
# the solution is in the following format
## col, row
## x,y

Fed to homography and resulted from homography:
------------------------------------------------
old value of A =  [0, 0, 1] new value =  [458.  73.   1.]
old value of B =  [0, 682, 1] new value =  [489. 682.   1.]
old value of C=  [1023, 0, 1] new value =  [1.587e+03 0.000e+00 1.000e+00]
old value of D =  [1023, 682, 1] new value =  [1.691e+03 6.820e+02 1.000e+00]


In [18]:
min_x , max_x , min_y, max_y = Get_max_min_Axis (A,B,C,D)
print("min_y =", min_y , "max_y = ", max_y ,"min_x = ", min_x,"max_x=", max_x)

min_y = 0.0 max_y =  682.0 min_x =  458.0 max_x= 1691.0


In [19]:
new_h = int(round(abs(max_y - min_y)))
new_w = int(round(abs(max_x- min_x)))
print("The new width is (# no of columns) =",new_w)
print("The new height is (# no of rows) =",new_h)

The new width is (# no of columns) = 1233
The new height is (# no of rows) = 682


## Inverse warping

In [20]:
#compute inverse homography
h_inv = np.linalg.inv(h)
h

array([[ 6.98500651e-01,  1.91202221e-02,  4.57640096e+02],
       [-1.39065903e-01,  8.93365607e-01,  7.26305084e+01],
       [-2.55504204e-04, -5.55418468e-05,  1.00000000e+00]])

In [21]:
#prepare x and y arrays for interpolation
#image 2 because inverse warping is made so that we get image 2 pixel values.
x = np.arange(0, img2.shape[1], 1)
y = np.arange(0, img2.shape[0], 1)

In [35]:
def warp_image(src_img,new_borders,h_inv):
    
    blue = src_img[:, :, 0] 
    green =  src_img[:, :, 1] 
    red= src_img[:, :, 2] 
    
    min_x= dimensions['min_x']
    min_y= dimensions['min_y']
    new_w= dimensions['new_w']
    new_h= dimensions['new_h']
    
    
    new_img_red, new_img_blue,new_img_green= np.zeros((src_img.shape[0],new_h+src_img.shape[1]))
    new_img_blue=np.zeros((src_img.shape[0],new_h+src_img.shape[1]))
    new_img_green=np.zeros((src_img.shape[0],new_h+src_img.shape[1]))

    interp_spline_green = RectBivariateSpline(y.reshape(-1,1), x.reshape(-1,1), green)
    interp_spline_red = RectBivariateSpline(y.reshape(-1,1), x.reshape(-1,1), red)
    interp_spline_blue= RectBivariateSpline(y.reshape(-1,1), x.reshape(-1,1), blue)

    ##start warping loop
    for i in range (int(min_x),int(min_x+new_w),1):
        for j in range (int(min_y),int((min_y+new_h)),1):
            point = [i,j,1]
            co_ord = np.matmul(h_inv,point)
            co_ord = correct(co_ord)
            xi=co_ord[0]
            yi=co_ord[1]
            if (xi>=0 and yi>=0):

                intensity=int(interp_spline_red.ev(yi, xi))
                new_img_red[j][i] = intensity


                intensity=int(interp_spline_blue.ev(yi, xi))
                new_img_blue[j][i] = intensity

                intensity=int(interp_spline_green.ev(yi, xi))
                new_img_green[j][i] = intensity



            else:
                new_img_red[j][i] = 0
                new_img_green[j][i] = 0
                new_img_blue[j][i] = 0
                
    return new_img_red,new_img_green,new_img_blue

    

In [36]:
dimensions = {}
dimensions['min_x']= min_x
dimensions['min_y'] = min_y
dimensions['new_w']= new_w
dimensions['new_h']=new_h

new_img_red,new_img_green,new_img_blue = warp_image(img2,dimensions,h_inv)

ValueError: too many values to unpack (expected 3)

In [33]:

figure(2)
#initialize the new image that will sticth the two images and the merge the bgr warped channels.
new_img = np.zeros((img2.shape[0],img2.shape[1]+new_h,3), 'uint8')
new_img[:,:,0]=new_img_blue
new_img[:,:,1]=new_img_green
new_img[:,:,2]=new_img_red

plt.imshow(new_img),
plt.show()
plt.close()

# Stitch the two images together

In [34]:
new_img[0:img1.shape[0],0:img1.shape[1]]= img1
plt.imshow(new_img,cmap='gray')
plt.show()
plt.close()

## Test using open cv functions

In [ ]:
dst = cv2.warpPerspective(img2,np.array(h),(img1.shape[1] + img2.shape[1], img1.shape[0]))
print(dst.shape)
figure(2)
#plt.imshow(dst,cmap='gray')
#plt.show()
#plt.close()
dst[0:img1.shape[0], 0:img1.shape[1]] = img1
plt.imshow(dst,cmap='gray')
plt.show()
plt.close()